<a href="https://colab.research.google.com/github/blufzzz/Dynamic-Texture-3D-CNN/blob/master/3D_CNN_Video_AR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook is demo for dynamic texture synthesis via auto-regresion of the frame sequence

In [1]:
import urllib
import requests
from os import path
# нужна информация о нашей системе
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

# уточним на чем будем вычислять
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x562d0000 @  0x7eff8bb0b2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
fastai 1.0.41 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
torch version 0.4.1
torchvision version 0.2.1
is cuda availible True


In [15]:
!pip install --no-cache-dir -I pillow

    100% |████████████████████████████████| 2.0MB 79.5MB/s 
fastai 1.0.41 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  [PIL]
You must restart the runtime in order to use newly installed versions.


In [0]:
from skimage import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.core.display import clear_output
import torch
import torchvision
from torch import nn


In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd ./gdrive/My\ Drive/Colab Notebooks/two-stream-dyntex-synth

/content/gdrive/My Drive/Colab Notebooks/two-stream-dyntex-synth


In [5]:
class CNN3D(nn.Module):
  
  def __init__(self):
    
    super(CNN3D).__init__()
  
    self.model = nn.Sequential(
                  nn.Conv3d(3,64, 3, stride=2, padding = (0, 2, 2))
                  nn.MaxPool3d()
                  nn.LeakyReLU()
                  nn.Conv3d(64, 64, 3, stride = 2)
                  nn.MaxPool3d()
                  nn.LeakyReLU()
                  nn.Conv3d(64,84,3, stride = 2)
                  nn.MaxPool3d()
                  nn.LeakyReLU()  
                  
                  nn.MaxUnpool3d()
                  nn.ConvTranspose3d()
                  nn.LeakyReLU()
                  nn.MaxUnpool3d()
                  nn.ConvTranspose3d()
                  nn.LeakyReLU()
                  nn.MaxUnpool3d()
                  nn.ConvTranspose3d()
                  nn.LeakyReLU()  
                )

    
  def forward(self, tensor):
    
    return self.model(tensor)

SyntaxError: ignored

In [51]:
img1 = io.imread('./data/dynamic_textures/fur/frame_00000000.png').transpose(2,0,1)
img2 = io.imread('./data/dynamic_textures/fur/frame_00000001.png').transpose(2,0,1)
x = np.array([[img1,img2, img1, img2]])

x = torch.Tensor(x).transpose(2,1)
x.shape

torch.Size([1, 3, 4, 256, 256])

In [0]:
conv = nn.Conv3d(3,64, 3, stride=1) # , padding = (0, 2, 2)
pool = nn.MaxPool3d((1,3,3), stride = 1)

In [53]:
pool(x).shape

torch.Size([1, 3, 4, 254, 254])

In [54]:
c = conv(x)
c.shape

torch.Size([1, 64, 2, 254, 254])